In [1]:
pip install torch numpy transformers datasets tiktoken wandb tqdm hazm

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
! git clone https://github.com/karpathy/nanoGPT.git

Cloning into 'nanoGPT'...
remote: Enumerating objects: 682, done.
remote: Total 682 (delta 0), reused 0 (delta 0), pack-reused 682
Receiving objects: 100% (682/682), 952.47 KiB | 20.71 MiB/s, done.
Resolving deltas: 100% (385/385), done.


In [3]:
cd nanoGPT

/content/nanoGPT


In [4]:
!mkdir data/poem

In [5]:
import pandas as pd

In [45]:
data5

,body
0,آن دل که شد او قابل انوار خدا
1,پر باشد جان او ز اسرار خدا
2,زنهار تن مرا چو شمع تنها مشمر
3,کو جمله به نمک زار خدا
4,آن شمع رخ تو لگنی نیست بیا
...,...
4395,عشق من و معشوق مرا نیست زوال
4396,عمری به هوس در تک و تاز آمد دل
4397,تا محرم جان دلنواز آمد دل
4398,در آخر کار رفت و جان پاک بسوخت


In [41]:
data1 = pd.read_csv('../khayyam_norm.txt', names=['body'])
data2 = pd.read_csv('../abusaeed_norm.txt', names=['body'])
data3 = pd.read_csv('../roodaki_norm.txt', names=['body'])
data4 = pd.read_csv('../hatef_norm.txt', names=['body'])
data5 = pd.read_csv('../moulavi_norm.txt', names=['body'])

data = pd.concat([data1, data2, data3, data4, data5])

special_tokens = ["<|startofverse|>", "<|endofverse|>", "<|startofstanza|>", "<|endofstanza|>"]
data.body = data.body.apply(lambda x:  f"<|startofverse|> {x} <|endofverse|>")
data

,body
0,<|startofverse|> برخیز و بیا بتا برای دل ما <|...
1,<|startofverse|> حل کن به جمال خویشتن مشکل ما ...
2,<|startofverse|> یک کوزه شراب تا بهم نوش کنیم ...
3,<|startofverse|> زان پیش که کوزه ها کنند از گل...
4,<|startofverse|> چون عهده نمی شود کسی فردا را ...
...,...
4395,<|startofverse|> عشق من و معشوق مرا نیست زوال ...
4396,<|startofverse|> عمری به هوس در تک و تاز آمد د...
4397,<|startofverse|> تا محرم جان دلنواز آمد دل <|e...
4398,<|startofverse|> در آخر کار رفت و جان پاک بسوخ...


In [46]:
# Function to concatenate rows
def concat_rows(group):
    return pd.Series({
        'body': '\n'.join(group['body']),
    })

# Concatenate every 4 rows
result = data.groupby(data.index // 4).apply(concat_rows).reset_index(drop=True)
result.body = result.body.apply(lambda x: "<|startofstanza|>" + x + "<|endofstanza|>")

print(result)

result.to_csv('data/poem/poem.txt', index=False)

                                                   body
0     <|startofstanza|><|startofverse|> برخیز و بیا ...
1     <|startofstanza|><|startofverse|> چون عهده نمی...
2     <|startofstanza|><|startofverse|> قرآن که مهین...
3     <|startofstanza|><|startofverse|> گر می نخوری ...
4     <|startofstanza|><|startofverse|> هر چند که رن...
...                                                 ...
1095  <|startofstanza|><|startofverse|> در خاموشی چر...
1096  <|startofstanza|><|startofverse|> در عشق نوا ج...
1097  <|startofstanza|><|startofverse|> عشقی به کمال...
1098  <|startofstanza|><|startofverse|> عشقی دارم پا...
1099  <|startofstanza|><|startofverse|> عمری به هوس ...

[1100 rows x 1 columns]


In [47]:
! python data/poem/prepare.py

train has 408,905 tokens
val has 44,127 tokens


In [33]:
! cp config/train_shakespeare_char.py config/train_peom.py

In [ ]:
import math

def calculate_small_gpt_parameters(total_tokens, unique_tokens, compute_budget):
    """
    Calculate optimal GPT parameters for a small model based on modified Chinchilla scaling laws.

    Args:
    - total_tokens (int): Total number of tokens in the dataset.
    - unique_tokens (int): Total number of unique tokens (vocabulary size).
    - compute_budget (float): Total compute budget in FLOPs.

    Returns:
    - dict: Dictionary containing optimal GPT parameters.
    """
    # Modified Chinchilla scaling heuristic for token-to-parameter ratio (increased for smaller models)
    token_to_param_ratio = 40

    # Estimate the optimal model size
    optimal_model_size = math.sqrt(compute_budget / token_to_param_ratio)

    # Set tighter bounds for the parameters
    max_n_embd = 512   # Upper limit for embedding size in smaller models
    min_n_embd = 64    # Lower limit for embedding size in smaller models

    max_n_layer = 12   # Upper limit for layers in smaller models
    min_n_layer = 2    # Lower limit for layers in smaller models

    # Adjust the scaling to fit within the bounds
    scaling_factor = 1e10  # Increased to further reduce model size

    # Set the model parameters based on the optimal model size
    n_embd = int(min_n_embd + (optimal_model_size / scaling_factor) * (max_n_embd - min_n_embd))
    n_embd = max(min_n_embd, min(max_n_embd, n_embd))  # Ensure n_embd is within bounds

    n_layer = int(min_n_layer + (optimal_model_size / scaling_factor) * (max_n_layer - min_n_layer))
    n_layer = max(min_n_layer, min(max_n_layer, n_layer))  # Ensure n_layer is within bounds

    # Ensure n_embd is a multiple of n_head
    n_head = max(2, n_embd // 64)  # Ensure at least 1 head and n_embd is divisible by n_head
    n_embd = (n_embd // n_head) * n_head

    # fetch vocab_size for tokenizer
    vocab_size = tokenizer.vocab_size

    # Calculate approximate number of parameters
    num_params = n_embd * n_layer * (4 * n_embd + vocab_size) / 1e6  # in millions

    # Return the calculated parameters
    return {
        "n_embd": n_embd,
        "n_head": n_head,
        "n_layer": n_layer,
        "vocab_size": vocab_size,
        "block_size": 32,  # Reduced context size for smaller models
        "approx_params_millions": round(num_params, 2)
    }

# Example usage
compute_budget = 1e18  # Reduced compute budget for smaller models

gpt_parameters = calculate_small_gpt_parameters(408905, unique_tokens, compute_budget)
print(gpt_parameters)

In [50]:
!python train.py config/train_peom.py --device=cuda --compile=False --eval_iters=20 --log_interval=1 --block_size=64 --batch_size=128 --n_layer=4 --n_head=4 --n_embd=72 --max_iters=10000 --lr_decay_iters=500 --dropout=0.0

Streaming output truncated to the last 5000 lines.
iter 5030: loss 1.1710, time 190.91ms, mfu 0.32%
iter 5031: loss 1.1377, time 192.61ms, mfu 0.32%
iter 5032: loss 1.1507, time 189.96ms, mfu 0.32%
iter 5033: loss 1.1292, time 192.71ms, mfu 0.32%
iter 5034: loss 1.1289, time 190.24ms, mfu 0.32%
iter 5035: loss 1.1096, time 192.45ms, mfu 0.32%
iter 5036: loss 1.1179, time 190.27ms, mfu 0.32%
iter 5037: loss 1.1448, time 193.77ms, mfu 0.32%
iter 5038: loss 1.1389, time 191.08ms, mfu 0.32%
iter 5039: loss 1.1340, time 189.24ms, mfu 0.32%
iter 5040: loss 1.0982, time 196.56ms, mfu 0.32%
iter 5041: loss 1.1375, time 188.60ms, mfu 0.32%
iter 5042: loss 1.1251, time 202.03ms, mfu 0.32%
iter 5043: loss 1.1236, time 189.65ms, mfu 0.32%
iter 5044: loss 1.1033, time 201.68ms, mfu 0.32%
iter 5045: loss 1.1201, time 182.75ms, mfu 0.32%
iter 5046: loss 1.1320, time 197.15ms, mfu 0.32%
iter 5047: loss 1.1092, time 185.34ms, mfu 0.32%
iter 5048: loss 1.1426, time 194.97ms, mfu 0.32%
iter 5049: loss 1.

In [53]:
!python sample.py --out_dir=out-poem --device=cuda

Overriding: out_dir = out-poem
Overriding: device = cuda
number of parameters: 3.87M
No meta.pkl found, assuming GPT-2 encodings...

<|startofverse|> کاین عاقع کنای تو کاندر روز <|endofverse|>
<|startofverse|> من دستم که ز دلمان قدح بادام <|endofverse|>
<|startofverse|> سرخ و ز خود در مشک می آید کرد کن <|endofverse|>
<|startofverse|> در زنده و خروشست کوتان شدیم <|endofverse|>
<|startofverse|> بختم برون عالم بیاید توانم <|endofverse|>
<|startofverse|> بیزد که درین رو زهرهای راست <|endofverse|>
<|startofverse|> خیر در مشک جهان می از من فتحست <|endofverse|>
<|startofverse|> آنکس گل آن روز بمها مه بید <|endofverse|>
<|startofverse|> جانی که زلف تو بر دستم بکشد <|endofverse|>
<|startofverse|> بر من دستم خویش من در آمدی <|endofverse|>
<|startofverse|> می باشد که شم خورد بی�
---------------

<|startofverse|> بی زان جان نبد شود است شکند <|endofverse|>
<|startofverse|> این دیده گریه روی جان شود <|endofverse|>
<|startofverse|> پنج بازد از دل ناله بردی <|endofverse|>
<|startofverse|> بر من مرا رن

In [55]:
!python sample.py --out_dir=out-poem --device=cuda --start='<|startofstanza|><|startofverse|>'

Overriding: out_dir = out-poem
Overriding: device = cuda
Overriding: start = <|startofstanza|><|startofverse|>
number of parameters: 3.87M
No meta.pkl found, assuming GPT-2 encodings...
<|startofstanza|><|startofverse|> ایزدی که بر لعلت باز دو تو <|endofverse|>
<|startofverse|> زان ما تنگ بوی شب بی نمامه نشود <|endofverse|>
<|startofverse|> جان را توام ز خوش بگشاید آن <|endofverse|>
<|startofverse|> گر زانکه شدی در و خروشست را <|endofverse|><|endofstanza|>
<|startofstanza|><|startofverse|> بی من و عالم بی دستی شکسته از شد <|endofverse|>
<|startofverse|> دیریک و نوش میخورم است و نانک <|endofverse|>
<|startofverse|> بر جهان نگشاید که زند شود <|endofverse|><|endofstanza|>
<|startofstanza|><|startofverse|> در های نگوند قال شکسته است از آن <|endofverse|>
<|startofverse|> با کامریست که بی در دلم نگر <|endofverse|>
<|startofverse|> با لب نباشته ای می باشد این <|endofverse|>
<|startof
---------------
<|startofstanza|><|startofverse|> غم شب که دلم بهانه مقصود است <|endofverse|>
<|startofverse|>

In [ ]:
!python train.py config/train_peom.py --device=cuda --compile=False --eval_iters=20 --log_interval=1 --block_size=16 --batch_size=128 --n_layer=4 --n_head=4 --n_embd=128 --max_iters=5000 --lr_decay_iters=500 --dropout=0.0

Streaming output truncated to the last 5000 lines.
iter 39: loss 8.3596, time 52.30ms, mfu 0.54%
iter 40: loss 8.2671, time 51.55ms, mfu 0.54%
iter 41: loss 8.1479, time 54.67ms, mfu 0.54%
iter 42: loss 8.0571, time 54.19ms, mfu 0.54%
iter 43: loss 7.9410, time 60.45ms, mfu 0.53%
iter 44: loss 7.8223, time 46.34ms, mfu 0.54%
iter 45: loss 7.7089, time 52.76ms, mfu 0.54%
iter 46: loss 7.6084, time 65.60ms, mfu 0.53%
iter 47: loss 7.4645, time 45.12ms, mfu 0.54%
iter 48: loss 7.3493, time 54.33ms, mfu 0.54%
iter 49: loss 7.2113, time 52.55ms, mfu 0.54%
iter 50: loss 7.0824, time 54.26ms, mfu 0.54%
iter 51: loss 6.9491, time 54.54ms, mfu 0.54%
iter 52: loss 6.8323, time 51.52ms, mfu 0.54%
iter 53: loss 6.6788, time 56.79ms, mfu 0.53%
iter 54: loss 6.5280, time 53.25ms, mfu 0.53%
iter 55: loss 6.3836, time 52.59ms, mfu 0.54%
iter 56: loss 6.2619, time 54.04ms, mfu 0.53%
iter 57: loss 6.0876, time 52.10ms, mfu 0.54%
iter 58: loss 5.9471, time 58.09ms, mfu 0.53%
iter 59: loss 5.8100, time 50

In [ ]:
!python sample.py --out_dir=out-poem --device=cuda

Overriding: out_dir = out-poem
Overriding: device = cuda
number of parameters: 7.23M
No meta.pkl found, assuming GPT-2 encodings...

چنین به چون دار مرد سرایش ز کند نیک سیاه
که تنگست از بر نیاز بارک دشمن اندرون پیل و جنگ آمد ایزد سراب
ز داری نام کجا بیاشست بهام به تاج زالین رنگ جنگ این شهر به پیش شود رسید که بر هرگز را دست سرگشست که باشد نبرد به آگینی
نگه پیلان گویم
نگهشت پیش خویش به دل شنگل پیاده جویدست بهرام زمین
که نیز این بر نهادم ببه بیامد بدو نیک که به داد آمد از تو کشته این گفت برو برافشان
جهدار چو توران سرایهی الجوین روشن سپهر
ازان سوی دست بهرام �
---------------

همی گفت کای درود
چنین داد پاسخ که بر پیکر اندر آید کنند آگهی را بدو فراوان آنک جامه برو بازگبان بخندید گردد برو سپهر دژ سوی کنون بود بگزید پیش ز گذر
چو داشت بهرام خوانی بارگاه ام بود
به شاهی شیر من نیکست که زینها ایرانیان کاین که دیوان و پیران گشته بر خواب کنجمن تهمتن بان بیامد کسی را جوانمرد براند آن بی نرم به جوی کینه بر درد تیرهی شاد
زن تخم شاه ز گرد گپ وگر نیک فرمیده کان را بخواند بر اندرون به گردانی تاج مرا بار
چ

In [ ]:
!python train.py config/train_peom.py --device=cuda --compile=False --eval_iters=20 --log_interval=1 --block_size=32 --batch_size=64 --n_layer=4 --n_head=4 --n_embd=128 --max_iters=5000 --lr_decay_iters=500 --dropout=0.0

Streaming output truncated to the last 5000 lines.
iter 39: loss 8.3787, time 52.11ms, mfu 0.54%
iter 40: loss 8.2768, time 53.94ms, mfu 0.54%
iter 41: loss 8.1845, time 50.57ms, mfu 0.54%
iter 42: loss 8.0794, time 53.06ms, mfu 0.54%
iter 43: loss 7.9559, time 50.78ms, mfu 0.54%
iter 44: loss 7.8379, time 52.74ms, mfu 0.54%
iter 45: loss 7.7485, time 51.54ms, mfu 0.55%
iter 46: loss 7.6086, time 51.47ms, mfu 0.55%
iter 47: loss 7.4895, time 55.29ms, mfu 0.54%
iter 48: loss 7.3563, time 49.31ms, mfu 0.55%
iter 49: loss 7.2331, time 54.08ms, mfu 0.55%
iter 50: loss 7.1365, time 53.23ms, mfu 0.54%
iter 51: loss 6.9812, time 51.56ms, mfu 0.55%
iter 52: loss 6.8591, time 51.91ms, mfu 0.55%
iter 53: loss 6.7122, time 52.07ms, mfu 0.55%
iter 54: loss 6.5714, time 52.77ms, mfu 0.55%
iter 55: loss 6.4192, time 51.13ms, mfu 0.55%
iter 56: loss 6.2502, time 52.91ms, mfu 0.55%
iter 57: loss 6.1172, time 53.11ms, mfu 0.55%
iter 58: loss 5.9736, time 50.21ms, mfu 0.55%
iter 59: loss 5.8124, time 53

In [ ]:
!python sample.py --out_dir=out-poem --device=cuda

Overriding: out_dir = out-poem
Overriding: device = cuda
number of parameters: 7.23M
No meta.pkl found, assuming GPT-2 encodings...

بدو گفت که گاه مرا سپاه را گرفت روی سیاه روان پست بویند به روی سخت بشنید با یکی پرستنده توان ز خود بر مشک که را
چو خورش سخن طاسان به تاج و پیر خویش
سپاهی شهریار چانکی بر برختر بر هرگشت گفتی شاه گرفته خواستند آن به گردن
ننگام ز گیتی باید شب کاه هر بویم به مرز تنگ
فگند که گس زین سازی گرفت بهرام برخاست مهترم ببه بدند بر چون از لشکر یاد
برین دشت چو بخواندند مرا را به جای درگاهدی
چو تاره سوی بهیوار آن دیدم زین خسروانی چو آن به دی
---------------

همی گنگ روز گردد سر گردن که آمد سپهبد پر زندها دلم رنج خنگ برآیدت بدین را گرانمای مرا
که بازد ایران برآتختند چون سیستی بر موی کنار بنشست آسود و پیویش اندر به کوی به گرد خریم
به تیری بوشت کار سپاه همه جویده سر سپودر به بر چهر
اگر آمد آن زین پس پیش و گیرد و گیتیم که گنه رزم بودی
که ای مرز جوانمان بران بتوهید فرو به مامید گنباه
چنان تیر پارس به کین بخویش بر سوی فرهادر کران
بیابی حخت زنخوانی بر چه گاه که رفتی آورده آورد چ